In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
pd.set_option('display.max_colwidth', 500)

In [15]:
pd.set_option('display.max_columns', 500)

In [3]:
data = pd.read_csv('../data/raw/combined_df_age_range_tags_spa.csv')

In [4]:
df = data.copy()

In [5]:
# sample_df = df.iloc[17:21,:]

In [6]:
# # Sample function to extract city/zone
# def extract_city(address):
#     if pd.isna(address):
#         return None
#     # Try to extract city name in parentheses first
#     if "(" in address and ")" in address:
#         return address.split("(")[-1].replace(")", "").strip()
#     # If no parentheses, take the last part after the comma
#     elif "," in address:
#         return address.split(",")[-1].strip()
#     else:
#         return None

# # Apply to the DataFrame
# df['city_zone'] = df['venue_address'].apply(extract_city)


In [ ]:
# Configurar el geolocalizador
geolocator = Nominatim(user_agent="city_locator")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Caché para coordenadas ya consultadas
city_cache = {}

# Función para obtener la ciudad
def get_city(lat, lon):
    key = (round(lat, 4), round(lon, 4))  # El redondeo mejora la tasa de aciertos de caché
    if pd.isna(lat) or pd.isna(lon):
        return None
    if key in city_cache:
        return city_cache[key]
    try:
        location = reverse((lat, lon), language='en')
        if location and 'address' in location.raw:
            address = location.raw['address']
            city = (
                address.get('city') or
                address.get('town') or
                address.get('village') or
                address.get('municipality')
            )
            city_cache[key] = city
            return city
    except Exception:
        city_cache[key] = None
        return None

mask = df['latitude'].notna() & df['longitude'].notna()

df.loc[mask, 'city'] = df.loc[mask].apply(
    lambda row: get_city(row['latitude'], row['longitude']), axis=1
)


In [11]:
df.shape

(315, 22)

In [9]:
# mask = sample_df['latitude'].notna() & sample_df['longitude'].notna()

# sample_df.loc[mask, 'city'] = sample_df.loc[mask].apply(
#     lambda row: get_city(row['latitude'], row['longitude']), axis=1
# )


------------------------------------------

Generando columna de ciudades para el csv con la base de datos completa descargada de Render

In [7]:
df = pd.read_csv('../data/raw/complete_data_db.csv')

In [8]:
df

,event_id,name,url,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,...,summary,is_online_event,tickets_url,tags,categoria,data_source,image,age_range,min_age,max_age
0,64,Pola Park,https://www.polapark.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
1,65,Parque de Atracciones de Zaragoza,https://www.atraczaragoza.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
2,66,Zoo de Barcelona,https://www.zoobarcelona.cat/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
3,67,Terra Mítica,https://www.terramiticapark.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
4,68,Aquópolis Cartaya,https://www.aquopolis.es/cartaya/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,12759320,¡Soy el chico/a del tiempo!,http://www.madrid.es/sites/v/index.jsp?vgnextc...,12:00,2025-05-31,NaN,2025-05-31,NaN,Centro de Educación Ambiental y Cultural Maris...,CALLE DOCTOR TOLOSA LATOUR 16,...,Horario: ﻿de 12 a 13 horas. Edad: Familias co...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,+6,6.0,14.0
1022,12751238,¿Hay alguien más aburrido que yo?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-09,NaN,2025-05-09,NaN,Biblioteca Pública Municipal Pablo Neruda (Ciu...,CALLE ASCAO 4,...,Una historia divertida y tierna sobre el poder...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,+6,6.0,14.0
1023,12762235,¿Jugamos con Europa...?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-08,NaN,2025-05-08,NaN,Biblioteca Pública Municipal La Chata (Caraban...,CALLE GENERAL RICARDOS 252,...,"La biblioteca celebra el Día de Europa, jugand...",NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,+8,8.0,14.0
1024,12762621,¿Qué hemos hecho en Madrid?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-17,NaN,2025-05-17,NaN,Parque Santa Eugenia,CALLE Cerro Almodóvar 1,...,Compañía Nos separa el puente y el río Ub...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,6-12,6.0,12.0


In [ ]:
# Configurar el geolocalizador
geolocator = Nominatim(user_agent="city_locator")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Caché para coordenadas ya consultadas
city_cache = {}

# Función para obtener la ciudad
def get_city(lat, lon):
    key = (round(lat, 4), round(lon, 4))  # El redondeo mejora la tasa de aciertos de caché
    if pd.isna(lat) or pd.isna(lon):
        return None
    if key in city_cache:
        return city_cache[key]
    try:
        location = reverse((lat, lon), language='en')
        if location and 'address' in location.raw:
            address = location.raw['address']
            city = (
                address.get('city') or
                address.get('town') or
                address.get('village') or
                address.get('municipality')
            )
            city_cache[key] = city
            return city
    except Exception:
        city_cache[key] = None
        return None

mask = df['latitude'].notna() & df['longitude'].notna()

df.loc[mask, 'city'] = df.loc[mask].apply(
    lambda row: get_city(row['latitude'], row['longitude']), axis=1
)


In [12]:
df.insert(10,'city', df.pop('city'))

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   event_id         1026 non-null   int64  
 1   name             1026 non-null   object 
 2   url              425 non-null    object 
 3   start_time       281 non-null    object 
 4   start_date       347 non-null    object 
 5   end_time         20 non-null     object 
 6   end_date         347 non-null    object 
 7   duration         20 non-null     object 
 8   venue_name       329 non-null    object 
 9   venue_address    926 non-null    object 
 10  city             686 non-null    object 
 11  longitude        686 non-null    float64
 12  latitude         686 non-null    float64
 13  summary          247 non-null    object 
 14  is_online_event  20 non-null     object 
 15  tickets_url      347 non-null    object 
 16  tags             1026 non-null   object 
 17  categoria     

In [16]:
df

,event_id,name,url,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,city,longitude,latitude,summary,is_online_event,tickets_url,tags,categoria,data_source,image,age_range,min_age,max_age
0,64,Pola Park,https://www.polapark.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Santa Pola,-0.553100,38.191000,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
1,65,Parque de Atracciones de Zaragoza,https://www.atraczaragoza.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zaragoza,-0.927800,41.634800,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
2,66,Zoo de Barcelona,https://www.zoobarcelona.cat/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Barcelona,2.192500,41.388400,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
3,67,Terra Mítica,https://www.terramiticapark.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Benidorm,-0.146800,38.563600,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
4,68,Aquópolis Cartaya,https://www.aquopolis.es/cartaya/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aljaraque,-7.005200,37.277500,NaN,NaN,NaN,ocio,Ocio,parks_list,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,12759320,¡Soy el chico/a del tiempo!,http://www.madrid.es/sites/v/index.jsp?vgnextc...,12:00,2025-05-31,NaN,2025-05-31,NaN,Centro de Educación Ambiental y Cultural Maris...,CALLE DOCTOR TOLOSA LATOUR 16,Madrid,-3.703222,40.374166,Horario: ﻿de 12 a 13 horas. Edad: Familias co...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,+6,6.0,14.0
1022,12751238,¿Hay alguien más aburrido que yo?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-09,NaN,2025-05-09,NaN,Biblioteca Pública Municipal Pablo Neruda (Ciu...,CALLE ASCAO 4,Madrid,-3.641690,40.429411,Una historia divertida y tierna sobre el poder...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,+6,6.0,14.0
1023,12762235,¿Jugamos con Europa...?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-08,NaN,2025-05-08,NaN,Biblioteca Pública Municipal La Chata (Caraban...,CALLE GENERAL RICARDOS 252,Madrid,-3.742063,40.383306,"La biblioteca celebra el Día de Europa, jugand...",NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,+8,8.0,14.0
1024,12762621,¿Qué hemos hecho en Madrid?,http://www.madrid.es/sites/v/index.jsp?vgnextc...,18:00,2025-05-17,NaN,2025-05-17,NaN,Parque Santa Eugenia,CALLE Cerro Almodóvar 1,Madrid,-3.608459,40.381311,Compañía Nos separa el puente y el río Ub...,NaN,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Niños,Ocio,ayuntamiento madrid,NaN,6-12,6.0,12.0


ultima versión del db, con todos los datos y columnas creadas hasta aquí

In [20]:
df.to_csv('../data/raw/complete_db_05_06_2025.csv', index=False)